In [1]:
# imports 
import numpy as np 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os 
import shutil
import random 
from random import sample 
import glob
import matplotlib.pyplot as plt 
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline


2022-08-31 17:23:25.686876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-31 17:23:25.686909: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_path = '/home/d/code/python/crack_detection/road_data/train/'
valid_path = '/home/d/code/python/crack_detection/road_data/valid/'
test_path = '/home/d/code/python/crack_detection/road_data/test/'

In [3]:
# orgnize data into directories

# 1) split into ok and broken 
fine_data = glob.glob("/home/d/code/python/crack_detection/road_trainer/*.fine.jpg")
broken_data = glob.glob("/home/d/code/python/crack_detection/road_trainer/*.broken.jpg")

fine_folder = "/home/d/code/python/crack_detection/road_data/train/fine/"
broken_folder = "/home/d/code/python/crack_detection/road_data/train/broken/"
validation_folder = "/home/d/code/python/crack_detection/road_data/valid/"

for image in fine_data:
    shutil.move(image, fine_folder)
for image in broken_data:
    shutil.move(image, broken_folder)

In [4]:
#                                  |  applies some preprocessint images-------------------------------|                    
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(1000, 668), classes=['broken', 'fine'], batch_size=2)

# shuffle is false on test batch, it so it's consistent with preprocessing  
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=validation_folder, target_size=(1000, 668), classes=['broken', 'fine'], batch_size=2, shuffle=False)

Found 1025 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [5]:
imgs, labels = next(train_batches)

In [6]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(1000, 668, 3)),
    MaxPool2D(pool_size=(2,2), strides=2), 
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same' ),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')])

2022-08-31 17:23:29.328047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-31 17:23:29.328795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-31 17:23:29.329260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-31 17:23:29.329538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-31 17:23:29.329905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.fit(x=train_batches, epochs=10, verbose=2)

Epoch 1/10


2022-08-31 17:23:30.628871: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192384000 exceeds 10% of free system memory.
2022-08-31 17:23:30.631967: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192384000 exceeds 10% of free system memory.
2022-08-31 17:23:31.655607: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192384000 exceeds 10% of free system memory.
2022-08-31 17:23:31.657791: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192384000 exceeds 10% of free system memory.
2022-08-31 17:23:32.710410: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192384000 exceeds 10% of free system memory.


In [ ]:
model.save("crack_detector.h5")